In [1]:
import csv
import io
import requests
import pandas as pd

import pyspark.sql.functions as F
from pyspark.sql import SparkSession
from pyspark.context import SparkContext
from pyspark.conf import SparkConf

from retrieve_data import DataRetrieverOverAPI

DAWUM_API_URL = "https://api.dawum.de/"

In [2]:
spark = SparkSession.builder \
                    .appName("Spark Basics") \
                    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/24 10:58:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
dawum_api = DataRetrieverOverAPI(DAWUM_API_URL)
data = dawum_api.data

In [4]:
dawum_database = data['Database']
dawum_parliaments = data['Parliaments']
dawum_institutes = data['Institutes']
dawum_taskers = data['Taskers']
dawum_methods = data['Methods']
dawum_parties = data['Parties']
dawum_surveys = data['Surveys']

print(type(dawum_surveys))

<class 'dict'>


In [5]:
df_database = spark.read.json(spark.sparkContext.parallelize([dawum_database]))
df_database = df_database.withColumn("license_name", F.col("License.Name")) \
                    .withColumn("license_shortcut", F.col("License.Shortcut")) \
                    .withColumn("license_link", F.col("License.Link")) \
                    .drop("License") \
                    .withColumnRenamed("Author", "author") \
                    .withColumnRenamed("Last_Update", "last_update") \
                    .withColumnRenamed("Publisher", "publisher") \
                    
df_database.show()


+--------------------+--------------------+---------+--------------------+----------------+--------------------+
|              author|         last_update|publisher|        license_name|license_shortcut|        license_link|
+--------------------+--------------------+---------+--------------------+----------------+--------------------+
|Dipl.-Jur. Philip...|2025-02-22T12:45:...| dawum.de|ODC Open Database...|        ODC-ODbL|https://opendatac...|
+--------------------+--------------------+---------+--------------------+----------------+--------------------+



In [6]:
rdd = spark.sparkContext.parallelize(dawum_parliaments.items())  # Parallelize the dictionary items

df_parliaments = rdd.map(lambda x: (x[0], x[1]['Name'], x[1]['Shortcut'], x[1]['Election'])).toDF(["parliament_id", "parliament_name", "parliament_shortcut", "parliament_election"])

df_parliaments.show()

+-------------+--------------------+--------------------+--------------------+
|parliament_id|     parliament_name| parliament_shortcut| parliament_election|
+-------------+--------------------+--------------------+--------------------+
|            0|           Bundestag|           Bundestag|      Bundestagswahl|
|            6|Hamburgische Bürg...|             Hamburg|Bürgerschaftswahl...|
|            9|Niedersächsischer...|       Niedersachsen|Landtagswahl in N...|
|            1|Landtag von Baden...|   Baden-Württemberg|Landtagswahl in B...|
|            8|Landtag von Meckl...|Mecklenburg-Vorpo...|Landtagswahl in M...|
|           16|Thüringischer Lan...|           Thüringen|Landtagswahl in T...|
|            4|Brandenburgischer...|         Brandenburg|Landtagswahl in B...|
|           15|Landtag von Schle...|  Schleswig-Holstein|Landtagswahl in S...|
|           14|Landtag von Sachs...|      Sachsen-Anhalt|Landtagswahl in S...|
|           13| Sächsischer Landtag|             Sac

In [7]:
rdd = spark.sparkContext.parallelize(dawum_institutes.items())  # Parallelize the dictionary items
df_institutes = rdd.map(lambda x: (x[0], x[1]["Name"])).toDF(["institute_id", "institute_name"])
df_institutes.show()

+------------+--------------------+
|institute_id|      institute_name|
+------------+--------------------+
|           5|                INSA|
|          24|Institut Wahlkrei...|
|          17|               Ipsos|
|           9|          Allensbach|
|           2|               Forsa|
|          13|              YouGov|
|          16|               Civey|
|           6|Forschungsgruppe ...|
|           1|     Infratest dimap|
|           4|                 GMS|
|          22|            pollytix|
|           3|      Verian (Emnid)|
|           7|Trend Research Ha...|
|          25|          IFM Berlin|
|          21|      Policy Matters|
|          18| Universität Hamburg|
|          15|         Mentefactum|
|          20|            IM Field|
|          12|              uniQma|
|          23|           Conoscope|
+------------+--------------------+
only showing top 20 rows



In [8]:
rdd = spark.sparkContext.parallelize(dawum_taskers.items())
df_taskers = rdd.map(lambda x: (x[0], x[1]["Name"])).toDF(["tasker_id", "tasker_name"])
df_taskers.show()

+---------+--------------------+
|tasker_id|         tasker_name|
+---------+--------------------+
|        4|                BILD|
|       97|Institut Wahlkrei...|
|       13|               Ipsos|
|        6|Frankfurter Allge...|
|       63|          RTL / n-tv|
|       43|              YouGov|
|      122|           Seven.One|
|        5|  ZDF-Politbarometer|
|       11|                 ARD|
|        7|                 GMS|
|       56| Schwäbische Zeitung|
|        3|     BILD am Sonntag|
|       10|     ARD-Tagesthemen|
|       67|            pollytix|
|       38|      Ostsee-Zeitung|
|       80|FUNKE Medien Thür...|
|       39|                 NDR|
|       64|               FOCUS|
|      112|Märkische Allgeme...|
|      120|Radio Hamburg / D...|
+---------+--------------------+
only showing top 20 rows



In [9]:
rdd = spark.sparkContext.parallelize(dawum_methods.items())
df_methods = rdd.map(lambda x: (x[0], x[1]["Name"])).toDF(["method_id", "method_name"])
df_methods.show()

+---------+----------------+
|method_id|     method_name|
+---------+----------------+
|        3|          Online|
|        2|      Persönlich|
|        1|     Telefonisch|
|        4|Telefon & Online|
|        0|       Unbekannt|
+---------+----------------+



In [10]:
rdd = spark.sparkContext.parallelize(dawum_parties.items())
df_parties = rdd.map(lambda x: (x[0], x[1]["Name"], x[1]["Shortcut"])).toDF(["party_id", "party_name", "party_shortcut"])
df_parties.show(df_parties.count())


+--------+--------------------+----------------+
|party_id|          party_name|  party_shortcut|
+--------+--------------------+----------------+
|       7|Alternative für D...|             AfD|
|      11|   Bayernpartei e.V.|    Bayernpartei|
|      14|Brandenburger Ver...|          BVB/FW|
|       4|Bündnis 90/Die Gr...|           Grüne|
|      23|Bündnis Sahra Wag...|             BSW|
|      21|bunt.saar – sozia...|       bunt.saar|
|      22|Bürger für Thüringen|            BfTh|
|      16|       Bürger in Wut|             BIW|
|       1|Christlich Demokr...|         CDU/CSU|
|     101|Christlich Demokr...|             CDU|
|     102|Christlich-Sozial...|             CSU|
|       5|           Die Linke|           Linke|
|      17|Familienpartei De...|         Familie|
|       3|Freie Demokratisc...|             FDP|
|       8|        Freie Wähler|    Freie Wähler|
|       9|Nationaldemokrati...|             NPD|
|      12|Ökologisch-Demokr...|             ÖDP|
|      13|Partei für

In [11]:
rdd = spark.sparkContext.parallelize(dawum_surveys.items())
df_surveys = rdd.map(lambda x: (x[0], 
                                x[1]["Parliament_ID"],
                                x[1]["Institute_ID"],
                                x[1]["Tasker_ID"],
                                x[1]["Method_ID"],
                                x[1]["Date"], 
                                x[1]["Survey_Period"]["Date_Start"], 
                                x[1]["Survey_Period"]["Date_End"],
                                x[1]["Surveyed_Persons"],
                                x[1]["Results"]
                                )) \
                        .toDF(["survey_id", 
                               "parliament_id",
                               "institute_id",
                               "tasker_id",
                               "method_id",
                               "survey_publish_date",
                               "survey_start_date", 
                               "survey_end_date",
                               "total_surveyees",
                               "results"])

df_surveys = df_surveys.select(F.col("survey_id"), 
                               F.col("parliament_id"),
                               F.col("institute_id"),
                               F.col("tasker_id"),
                               F.col("method_id"),
                               F.explode(df_surveys.results).alias("party_id", "survey_result_by_percent"),
                               F.col("survey_publish_date"),
                               F.col("survey_start_date"), 
                               F.col("survey_end_date"),
                               F.col("total_surveyees") 
                               )

df_surveys = df_surveys.fillna({"survey_result_by_percent": 0})
df_surveys.show(30)
df_surveys_3col = df_surveys.select("party_id", "survey_publish_date", "survey_result_by_percent", "institute_id")#.show(40)
df_surveys_3col.select("survey_publish_date").distinct().count()

+---------+-------------+------------+---------+---------+--------+------------------------+-------------------+-----------------+---------------+---------------+
|survey_id|parliament_id|institute_id|tasker_id|method_id|party_id|survey_result_by_percent|survey_publish_date|survey_start_date|survey_end_date|total_surveyees|
+---------+-------------+------------+---------+---------+--------+------------------------+-------------------+-----------------+---------------+---------------+
|     3774|            0|           5|        4|        3|       0|                     0.0|         2025-02-22|       2025-02-20|     2025-02-21|           2005|
|     3774|            0|           5|        4|        3|      23|                     0.0|         2025-02-22|       2025-02-20|     2025-02-21|           2005|
|     3774|            0|           5|        4|        3|       1|                    29.5|         2025-02-22|       2025-02-20|     2025-02-21|           2005|
|     3774|           

2065

In [12]:
df_surveys.join(df_parliaments, on="parliament_id" , how="full").join(df_parties, on="party_id", how="full").show()


+--------+-------------+---------+------------+---------+---------+------------------------+-------------------+-----------------+---------------+---------------+---------------+-------------------+-------------------+-----------------+--------------+
|party_id|parliament_id|survey_id|institute_id|tasker_id|method_id|survey_result_by_percent|survey_publish_date|survey_start_date|survey_end_date|total_surveyees|parliament_name|parliament_shortcut|parliament_election|       party_name|party_shortcut|
+--------+-------------+---------+------------+---------+---------+------------------------+-------------------+-----------------+---------------+---------------+---------------+-------------------+-------------------+-----------------+--------------+
|       0|            0|     3774|           5|        4|        3|                     0.0|         2025-02-22|       2025-02-20|     2025-02-21|           2005|      Bundestag|          Bundestag|     Bundestagswahl|sonstige Parteien|      So

In [13]:
print(df_database.columns)
print(df_parliaments.columns)
print(df_institutes.columns)
print(df_taskers.columns)
print(df_methods.columns)
print(df_parties.columns)
print(df_surveys.columns)

column_database = ['author', 'last_update', 'publisher', 'license_name', 'license_shortcut', 'license_link']
column_parliaments = ['parliament_id', 'parliament_name', 'parliament_shortcut', 'parliament_election']
column_institutes = ['institute_id', 'institute_name']
column_taskers = ['tasker_id', 'tasker_name']
column_methods = ['method_id', 'method_name']
column_parties = ['party_id', 'party_name', 'party_shortcut']
column_surveys = ['survey_id', 'parliament_id', 'institute_id', 'tasker_id', 'method_id', 'party_id', 'survey_result_by_percent', 'survey_publish_date', 'survey_start_date', 'survey_end_date', 'total_surveyees']

['author', 'last_update', 'publisher', 'license_name', 'license_shortcut', 'license_link']
['parliament_id', 'parliament_name', 'parliament_shortcut', 'parliament_election']
['institute_id', 'institute_name']
['tasker_id', 'tasker_name']
['method_id', 'method_name']
['party_id', 'party_name', 'party_shortcut']
['survey_id', 'parliament_id', 'institute_id', 'tasker_id', 'method_id', 'party_id', 'survey_result_by_percent', 'survey_publish_date', 'survey_start_date', 'survey_end_date', 'total_surveyees']


In [14]:
import sqlite3
conn = sqlite3.connect('./database/fun.db')

dfp_database = df_database.toPandas()
dfp_parliaments = df_parliaments.toPandas()
dfp_institutes = df_institutes.toPandas()
dfp_taskers = df_taskers.toPandas()
dfp_methods = df_methods.toPandas()
dfp_parties = df_parties.toPandas()
dfp_surveys = df_surveys.toPandas()

dfp_database.to_sql("", con=conn, if_exists="replace", index=False)
dfp_parliaments.to_sql("", con=conn, if_exists="replace", index=False)
dfp_institutes.to_sql("", con=conn, if_exists="replace", index=False)
dfp_taskers.to_sql("", con=conn, if_exists="replace", index=False)
dfp_methods.to_sql("", con=conn, if_exists="replace", index=False)
dfp_parties.to_sql("", con=conn, if_exists="replace", index=False)
dfp_surveys.to_sql("", con=conn, if_exists="replace", index=False)

OperationalError: unable to open database file

In [ ]:
pd.read_sql("select * from database_information;", con=conn)

,author,last_update,publisher,license_name,license_shortcut,license_link
0,Dipl.-Jur. Philipp Guttmann,2025-02-18T13:00:31+01:00,dawum.de,ODC Open Database License,ODC-ODbL,https://opendatacommons.org/licenses/odbl/1-0/
